In [11]:
from google.colab import files
uploaded = files.upload()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("forestfires.csv")
df.head()

Saving Problem_statement(fireforests).txt to Problem_statement(fireforests).txt


,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small


In [12]:
df = df.drop(['month', 'day'], axis=1)
df['size_category'] = df['size_category'].map({
    'small': 0,
    'large': 1
})

In [13]:
X = df.drop('size_category', axis=1)
y = df['size_category']


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(16, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [18]:
history = model.fit(
    X_train_scaled,
    y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=16
)


Epoch 1/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.6558 - loss: 0.6772 - val_accuracy: 0.7229 - val_loss: 0.6124
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7077 - loss: 0.6132 - val_accuracy: 0.7470 - val_loss: 0.5904
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7173 - loss: 0.6024 - val_accuracy: 0.7470 - val_loss: 0.5761
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7137 - loss: 0.6031 - val_accuracy: 0.7470 - val_loss: 0.5663
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7087 - loss: 0.5899 - val_accuracy: 0.7470 - val_loss: 0.5583
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7316 - loss: 0.5621 - val_accuracy: 0.7470 - val_loss: 0.5511
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7451 - loss: 0.5385 - val_accuracy: 0.7470 - val_loss: 0.5472
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7503 - loss: 0.5235 - val_accuracy: 0.7470 - val

In [19]:
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", accuracy)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7782 - loss: 0.7679
Test Accuracy: 0.7788461446762085


In [20]:
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


In [21]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[71  5]
 [18 10]]
              precision    recall  f1-score   support

           0       0.80      0.93      0.86        76
           1       0.67      0.36      0.47        28

    accuracy                           0.78       104
   macro avg       0.73      0.65      0.66       104
weighted avg       0.76      0.78      0.75       104



In [ ]:
#Fire indices such as FFMC, DMC, ISI strongly influence fire size
#Weather variables (temperature, humidity, wind) play a supporting role
#Neural Network successfully captures non-linear relationships